#### Import libraries and load environment

In [1]:
import os
import re
import json
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from typing import Optional
from langchain.chat_models import init_chat_model
from pydantic_settings import BaseSettings, SettingsConfigDict
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_community.vectorstores import FAISS
from langchain.embeddings import init_embeddings


def load_env():
	env_file_path = ".env"
	if os.path.exists(env_file_path):
		load_dotenv(env_file_path, override=True)
		print("Loaded environment variables")
	else:
		print(f"Error: .env file not found at {env_file_path}")

load_env()

class ModelSettings(BaseSettings):
	model: str
	temperature: Optional[float] = 0
	max_tokens: Optional[int] = None

class ProcessModelSettings(BaseSettings):
	gpt41: ModelSettings
	gpt4o_mini: ModelSettings
	gpt41_mini: ModelSettings
	o1_mini: ModelSettings
	gpt41_nano: ModelSettings
	deepseek: ModelSettings
	phi4: ModelSettings
	embedding: ModelSettings


class Settings(BaseSettings):
	model_config = SettingsConfigDict(env_file=".env", env_nested_delimiter="__")

	# Keys
	azure_openai_api_key: Optional[str]
	azure_openai_endpoint: Optional[str]
	openai_api_version: Optional[str]

	openai_api_deployment_name_gpt4_vision_no_filters: Optional[str]
	openai_vision_api_version_gpt4_vision_no_filters: Optional[str]
	openai_api_key_gpt4_vision_no_filters: Optional[str]
	openai_base_url_gpt4_vision_no_filters: Optional[str]

	# models
	processmodel: ProcessModelSettings


settings = Settings()

Loaded environment variables


#### Load rag vectorstore

In [4]:
embed_model = init_embeddings("ollama:nomic-embed-text:latest")
criteria_vectorstore = FAISS.load_local(folder_path="Data/VectorStores/Criteria_FAISS", embeddings=embed_model, allow_dangerous_deserialization=True)

def get_criteria_vectorstore(query, top_k=5):
	prefix = "search_query: "
	criteria_results = criteria_vectorstore.similarity_search(prefix + query, k=top_k)
	response = []
	for res in criteria_results:
		chap = res.metadata.get("chapter")
		response.append({"content": res.page_content, "chapter": chap})
	return response

#### Define prompt-template and llm call

In [5]:
semantic_prompt_template = PromptTemplate(
	template="""
You are an assistant for a medical triage system. Identify the 5 most relevant medical chapters based on the user query and chapter summaries. Each chapter has a numeric ID and a summary.

## Chapter Summaries:
{chapter_summaries}

## User Query:
{query}

# Instructions:
- Only select from the provided chapters.
- Do not hallucinate or invent any chapter names.
- Return exactly 5 chapter IDs in descending order of relevance.
- For chain of thought (cot), provide a bullet-point explanation for each selected candidate chapter, describing the reasoning behind its relevance to the query based on the provided summaries. Start each explanation with a dash (-) and place each on a new line.

# Important (enforce these rules):
- Prioritize **observations** and **cause** when evaluating a chapter's relevance.
- If the **cause** is marked as "unknown", rely on the **observations**.
- Only consider **condition** if it provides additional context beyond the patient's consciousness or breathing status.

# Output Format (example):

```json
{{
  "retrieved": ["05","38","01","10","07"]
  "cot": "Explaining the reasoning for the selection of these chapters based on the query and summaries provided for each selected candidate."
}}
```
""",
	input_variables=["query", "chapter_summaries"]
)



def call_llm_template(template, input_data):

	try:
		llm = init_chat_model(**settings.processmodel.gpt41.model_dump(exclude_none=True))
		chain = template | llm | StrOutputParser()
		return chain.invoke(input_data)
	except Exception as e:
		print(f"Error calling LLM: {e}")
	
	
	print("- Warning: Retrying with gpt4o-mini...")
	try:
		llm = init_chat_model(**settings.processmodel.gpt4o_mini.model_dump(exclude_none=True))
		chain = template | llm | StrOutputParser()
		return chain.invoke(input_data)
	except Exception as e:
		print(f"Error calling LLM: {e}")


	print("- Warning: Retrying with gpt41-mini...")
	try:
		llm = init_chat_model(**settings.processmodel.gpt41_mini.model_dump(exclude_none=True))
		chain = template | llm | StrOutputParser()
		return chain.invoke(input_data)
	except Exception as e:
		print(f"Error calling LLM: {e}")
	

	print("- Warning: Retrying with o1_mini...")
	try:
		llm = init_chat_model(**settings.processmodel.o1_mini.model_dump(exclude_none=True))
		chain = template | llm | StrOutputParser()
		return chain.invoke(input_data)
	except Exception as e:
		print(f"Error calling LLM: {e}")
	
	print("- Warning: Retrying with gpt41_nano...")
	try:
		llm = init_chat_model(**settings.processmodel.gpt41_nano.model_dump(exclude_none=True))
		chain = template | llm | StrOutputParser()
		return chain.invoke(input_data)
	except Exception as e:
		print(f"Error calling LLM: {e}")

#### Stages for retrieval
- combines rag (5) + semantic retrieval llm on chapter names (5)
- refines each 10 with context from short summary and reduces to 5 most relevant

In [9]:
def clean_chapter(text):
	text = text.replace("/", ",")
	text = re.sub(r'#', '', text)
	text = re.sub(r'\d+', '', text) 
	text = re.sub(r'\s*,\s*', ', ', text) 
	text = re.sub(r'\s+', ' ', text) 
	return text.lower().strip()

def extract_leading_number(text: str) -> str | None:
    match = re.match(r'^(\d+)', text.strip())
    return match.group(1) if match else None

def load_candidate_summaries(path):
	ids_candidate_mapping = {}
	ids_chapter_mapping = {}
	with open(path, "r", encoding="utf-8") as f:
		for line in f:
			try:
				entry = json.loads(line.strip())
				chapter = entry.get("chapter", "").strip()
				chapter_id = extract_leading_number(chapter)
				summary_text = entry.get("summary", "").strip()
				summary = f'Chapter: "{clean_chapter(chapter)}"\nSymptoms: {summary_text}'
				ids_candidate_mapping[chapter_id] = (chapter, clean_chapter(chapter))
				ids_chapter_mapping[chapter_id] = (chapter, summary)
			except json.JSONDecodeError as e:
				print(f"Skipping malformed line {id}: {e}")
	return ids_candidate_mapping, ids_chapter_mapping

def retrieve_candidate_chapters_prompt(query, debug=False):
	# RAG Retrieval
	retrieval = get_criteria_vectorstore(query, top_k=5)
	rag_candidates = [result["chapter"] for result in retrieval]
	
	# Semantic Retrieval LLM on chapter names
	ids_candidate_mapping, ids_chapter_mapping = load_candidate_summaries("Data/Chapter_Short_Summaries.jsonl")
	cleaned_chapters = {id: cleaned for id, (_, cleaned) in ids_candidate_mapping.items()}
	
	result = call_llm_template(semantic_prompt_template, {
		"query": query,
		"chapter_summaries":  json.dumps(cleaned_chapters, ensure_ascii=False, separators=(",", ":"))
	})

	candidate_ids = JsonOutputParser().parse(result)
	CoT = candidate_ids.get("cot", "")
	llm_chapters = [ids_candidate_mapping[str(id)][0] for id in candidate_ids["retrieved"]]
	
	if debug:
		display("🔍 RAG retrieval candidates:",rag_candidates)
		display("🔍 LLM retrieval candidates:",llm_chapters)
		print(f"🔍 Chain of Thought (CoT):\n{CoT}\n")

	# Combine candidates from RAG and LLM
	candidates = set(llm_chapters + rag_candidates)

	# Compare candidate summary content to find the most relevant chapter
	summaries = {id: summary for id, (chapter, summary) in ids_chapter_mapping.items() if chapter in candidates}

	result = call_llm_template(semantic_prompt_template, {
		"query": query,
		"chapter_summaries": json.dumps(summaries, ensure_ascii=False, separators=(",", ":"))
	})
	 
	chapter_ids = JsonOutputParser().parse(result)
	CoT = chapter_ids.get("cot", "")
	chapters = [ids_chapter_mapping[id][0] for id in chapter_ids["retrieved"]]

	if debug:
		display("🔍 Final selected chapters:",chapters)
		print(f"🔍 Final Chain of Thought (CoT): \n{CoT}\n")

	return chapters

In [10]:
from datasets import load_dataset
vague_dataset = load_dataset("json", data_files="Data/TestData/vague_data.jsonl", split="train")
print(f"vague_dataset\n{vague_dataset}")

input_texts = []
references = []

for entry in vague_dataset:
	input_texts.append(entry["input_text"].strip())
	references.append(entry["label"])

/home/erik/Desktop/multi_stage_retrieval/.venv/lib64/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


vague_dataset
Dataset({
    features: ['label', 'input_text'],
    num_rows: 35
})


In [13]:
def extract_first_number(text):
	match = re.search(r'\b\d+\b', text)
	if match:
		return match.group().strip()
	return ""

input_texts = []
references = []
combined_predictions = []
for entry in vague_dataset:
	input_texts.append(entry["input_text"].strip())
	references.append(entry["label"])


k = 5
input_file = "Data/Evaluation/RAG_vague.json"
with open(input_file, "r") as f:
	previous_data = json.load(f)


combined_candidate_predictions = []

for i, (input_text, label) in enumerate(zip(input_texts, references)):
	print(f"Processing entry {i+1}/{len(input_texts)}")
	chapters = retrieve_candidate_chapters_prompt(input_text, debug=True)
	candidates = [int(extract_first_number(chapter)) for chapter in chapters]

	combined_candidate_predictions.append({
		"normal":candidates[0],
		"best": label if label in candidates else candidates[0]
	})

rag_predictions = {
	"combined_candidate_predictions": combined_candidate_predictions
}

for key in previous_data:
	if key not in rag_predictions:
		rag_predictions[key] = previous_data[key]


# Save the data to a JSON file
output_file =  "./Data/Evaluation/COMBINED_RAG_vague.json"
with open(output_file, "w") as f:
    json.dump(rag_predictions, f, ensure_ascii=False, indent=2)

print(f"Inference results saved to {output_file}")

Processing entry 1/35


'🔍 RAG retrieval candidates:'

['36 Uncertain / unidentified problem',
 '24 Functional decline',
 '33 Breathing problems',
 '08 Drowning',
 '11 Accident / injury']

'🔍 LLM retrieval candidates:'

['03 Unconscious / decreased level of consciousness, breathing normally',
 '34 Drug abuse / intoxication / overdose',
 '30 Hyperthermia / heat stroke / heat exhaustion',
 '36 Uncertain / unidentified problem',
 '22 Fever / infection / sepsis']

...

['05 Mental health issue',
 '38 Ear, nose and throat',
 '36 Uncertain / unidentified problem',
 '19 Chest pain / cardiac disease',
 '32 Seizures (convulsions, fits)']

🔍 Chain of Thought (CoT):
- Chapter 05 (mental health issue): The person's confusion and tiredness could indicate a mental health issue, especially given the context of their altered state of consciousness and the unclear cause of their discomfort.
- Chapter 38 (ear, nose and throat): The symptoms of slight discoloration near the eyes, squinting, and discomfort on one side could suggest an issue related to the ear, nose, or throat, possibly indicating a sinus or other ENT-related problem.
- Chapter 36 (uncertain, unidentified problem): Since the cause of the discomfort is unclear, this chapter is relevant as it addresses situations where the problem is not easily identifiable, which aligns with the user's description.
- Chapter 19 (chest pain, cardiac disease): Although not directly mentioned, the overall symptoms of confusion and tiredness could potentially relate to cardiac issues, especially if there is an underlying condition that has not been identified.
- Chapter 32 (seizures - c

'🔍 Final selected chapters:'

['39 Eye',
 '28 Headache',
 '36 Uncertain / unidentified problem',
 '32 Seizures (convulsions, fits)',
 '05 Mental health issue']

🔍 Final Chain of Thought (CoT): 
- Chapter 39 (eye): The symptoms of slight discoloration near the eyes, squinting, and avoiding bright light suggest a potential eye issue, such as irritation or injury, which aligns with the chapter's focus on eye-related problems.
- Chapter 28 (headache): The mention of a sudden jolt and mild nausea could indicate a headache or concussion, especially given the confusion and tiredness, which are relevant to this chapter's symptoms.
- Chapter 36 (uncertain, unidentified problem): The overall confusion, tiredness, and unclear cause of discomfort fit well with this chapter, which addresses situations where the problem is not immediately identifiable.
- Chapter 32 (seizures): Although not explicitly mentioned, the confusion and unresponsiveness could suggest a seizure or postictal state, making this chapter relevant given the symptoms presented.
- Chapter 05 (mental health issue): While the primary symptoms do not directly indicate a mental health issue, t

### Using the retrieval
- Keep in mind debug is used to see CoT and results

In [ ]:
query =  {
	"condition": "Unconscious, not breathing normally, difficulty breathing",
	"observations": "Complaining about feeling ill for a couple of days",
	"cause": "Unknown medical condition",
	"callers_actions": "Helping patient"
}
query = json.dumps(query, indent=2, ensure_ascii=False)
chapter_ids = retrieve_candidate_chapters_prompt(query, debug=True)
print(chapter_ids)
# print(f"🧠 Chain Of Thought: {CoT}")
# display("🔐 Multi-Rank retrieval:",chapters)

'🔍 RAG retrieval candidates:'

['33 Breathing problems',
 '36 Uncertain / unidentified problem',
 '24 Functional decline',
 '16 Child / infant - illness',
 '05 Mental health issue']

'🔍 LLM retrieval candidates:'

['03 Unconscious / decreased level of consciousness, breathing normally',
 '33 Breathing problems',
 '22 Fever / infection / sepsis',
 '34 Drug abuse / intoxication / overdose',
 '05 Mental health issue']

🔍 Chain of Thought (CoT):
- Chapter 03 is highly relevant as it directly addresses the patient's unconscious state and decreased level of consciousness, which aligns with the query's condition of being unconscious and not breathing normally.
- Chapter 33 is pertinent because it covers breathing problems, which is a critical aspect of the patient's condition, especially given the difficulty in breathing mentioned in the query.
- Chapter 22 is relevant due to the observations of the patient feeling ill for a couple of days, which could indicate an underlying infection or sepsis, especially in the context of the patient's current state.
- Chapter 34 is included because it deals with drug abuse, intoxication, or overdose, which could be a potential cause of the patient's symptoms, especially since the cause is unknown.
- Chapter 05 is relevant as it addresses mental health issues, which could be a contributing factor to the patient's overall condition, particularly if the illness has psych

'🔍 Final selected chapters:'

['33 Breathing problems',
 '03 Unconscious / decreased level of consciousness, breathing normally',
 '34 Drug abuse / intoxication / overdose',
 '36 Uncertain / unidentified problem',
 '22 Fever / infection / sepsis']

🔍 Final Chain of Thought (CoT): 
- Chapter 33 is highly relevant because it specifically addresses breathing problems, which aligns with the user's condition of not breathing normally and difficulty breathing.
- Chapter 03 is also pertinent as it discusses unconsciousness and decreased levels of consciousness, which is a key aspect of the user's condition.
- Chapter 34 is relevant due to its focus on drug abuse and intoxication, which can lead to altered consciousness and breathing difficulties, especially given the unknown cause of the patient's condition.
- Chapter 36 is included because it deals with uncertain and unidentified problems, which fits the user's description of an unknown medical condition and the patient's unwell state.
- Chapter 22 is relevant as it covers symptoms of fever and infection, which could be underlying causes of the patient's complaints of feeling ill for a couple of days, potentially leading to the observed symptoms.

['33 Breathing problems', '03 Unconsci